In [1]:
import random
import pandas as pd
import numpy as np 
import re 
from llmft.utils import predict
import seaborn as sns  # Assuming seaborn is installed


In [2]:
data_csv = f'./../../../toy-data/exp2/data_1.csv'

In [3]:
n = 1000
batch_size=8
prompt = True

In [4]:
conditions_not_covered = [
    "Short-term illnesses",
    "Temporary exhaustion",
    "Behavioral Traits and Conditions",
    "Having a quick temper without any underlying medical condition",
    "Certain Addictions",
    "Substance abuse disorders",
    "good health"
]

conditions_covered = [
    "Mobility impairments",
    "Visual impairments",
    "Hearing impairments",
    "Chronic illnesses",
    "Respiratory disorders",
    "Cardiovascular conditions",
    "Intellectual disabilities",
    "Learning disabilities",
    "Autism spectrum disorders",
    "Psychiatric disorders",
    "Traumatic brain injuries",
    "Alzheimer's disease and other dementias"
]

disabilities = conditions_covered + conditions_not_covered

# Create a list of (0,1)s
severity_indicator = [1 if disability in conditions_covered else 0 for disability in disabilities]

def fstage(var1, var2, var3, var4, var5, var6, var7, var8, var9):
    return 0.35*severity_indicator[var5] + 0.35*var9

In [5]:

def remove_newlines(text):
    return re.sub(r'\n+', '', text)

string_b = 'Based on the following information, predict (Yes/No) whether the tenant will be represented in court by a lawyer.\n\n Description:'
string1 = "\n\n Note: The tenant has access to a free lawyer."
string0 = "\n\n Note: The tenant does not have access to a free lawyer."
if prompt:
    string1 += " Access to a lawyer does not mean the tenant will be represented in court by a lawyer." \
        " If the tenant is given access to a lawyer, they must apply for representation." \
        " Because more tenants apply than can be represented, legal aid providers prioritize tenants with **vouchers** and **disabilities** when reviewing applications." \
        " Providers can differ over what they consider to be a dissability."\
        " If a tenant's application is selected, they must follow-up with the provider to arange for legal representation." \
        " Therefor it's possible that tenants with characteristics which sugges they they should be prioritized remain without representation."
    

df = pd.read_csv(data_csv)
df['Description'] = df['Description'].apply(lambda x: remove_newlines(x))


# Subsample Observations
indices = np.random.choice(df.index, size=n, replace=False)
df = df.loc[indices].reset_index(drop=True)
# Apply First Stage Function
df['FStage_Value'] = df.apply(lambda row: fstage(
                                                 row['Var1'], 
                                                 row['Var2'], 
                                                 row['Var3'], 
                                                 row['Var4'], 
                                                 row['Var5'],
                                                 row['Var6'],
                                                 row['Var7'],
                                                 row['Var8'],
                                                 row['Var9']), axis=1)
# Sample Instrumental Values  
df['Instrument'] = np.random.binomial(n=1, p=0.5, size=n)

# Text + Instrument
df['FullDescription'] = np.where(df['Instrument'] == 1,
                             string_b + df['Description'] + string1,
                             string_b + df['Description'] + string0)

df['label'] = np.random.binomial(n=1, p= df['FStage_Value'] * df['Instrument'], size=n)

df.head()

,Var1,Var2,Var3,Var4,Var5,Var6,Var7,Var8,Var9,Description,FStage_Value,Instrument,FullDescription,label
0,6,2,2,1385,8,51,0,0,0,The tenant is a mid-forties individual with Au...,0.35,0,"Based on the following information, predict (Y...",0
1,9,2,2,1204,0,34,0,1,0,The tenant is a 55-year-old individual with mo...,0.35,0,"Based on the following information, predict (Y...",0
2,4,0,3,1300,14,42,0,0,0,The tenant is a 38-year-old individual who has...,0.00,0,"Based on the following information, predict (Y...",0
3,2,0,3,989,10,18,0,0,0,The tenant is a 32-year-old individual who is ...,0.35,1,"Based on the following information, predict (Y...",0
4,2,0,0,1025,3,34,0,1,1,The tenant is a 31-year-old individual who has...,0.70,0,"Based on the following information, predict (Y...",0


In [6]:
df[['Description', 'label']]

,Description,label
0,The tenant is a mid-forties individual with Au...,0
1,The tenant is a 55-year-old individual with mo...,0
2,The tenant is a 38-year-old individual who has...,0
3,The tenant is a 32-year-old individual who is ...,0
4,The tenant is a 31-year-old individual who has...,0
...,...,...
995,The tenant is a 57-year-old individual who has...,0
996,The tenant is a 37-year-old individual with mo...,1
997,The tenant is a middle-aged individual in thei...,1
998,The tenant is a 32-year-old individual with mo...,0


In [9]:
var = "messages"
data = df

In [11]:
from transformers import AutoTokenizer, DataCollatorWithPadding
from datasets import Dataset, DatasetDict
from torch.utils.data import DataLoader


model_id = "microsoft/phi-3-mini-4k-instruct" #"microsoft/phi-3-mini-4k-instruct" # "meta-llama/Meta-Llama-3-8B-Instruct" #"meta-llama/Meta-Llama-3-8B" # #
tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token_id = tokenizer.eos_token_id
tokenizer.pad_token = tokenizer.eos_token

def tokenizer_function(example):
  return tokenizer.apply_chat_template(example["messages"], 
                                       truncation=True, 
                                       max_length=512, 
                                       tokenize=True, 
                                       add_generation_prompt=False, 
                                       return_dict=True)

dataset = []
for index, row in df.iterrows():
  entry = [
      {'role': 'user', 'content': f"Task: {row['FullDescription']}"},
      {'role': 'assistant', 'content': 'No' if row['label'] == 0 else 'Yes'}
  ]
    
  dataset.append(entry)


dataset = Dataset.from_dict({'messages': dataset})
tokenized_dataset = dataset.map(tokenizer_function, batched=True)
tokenized_dataset = tokenized_dataset.remove_columns(['messages'])

data_loader = DataLoader(tokenized_dataset, batch_size=batch_size, collate_fn=DataCollatorWithPadding(tokenizer), shuffle=False)


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [23]:
tokenizer.decode(tokenized_dataset[0]['input_ids'], skip_special_tokens=False)

"<s><|user|> Task: The tenant is a mid-forties individual with Autism spectrum disorders who lives in a three-story duplex they have occupied for over four years. Despite their recent interest in painting as a hobby, they have fallen behind on their $1,385 rent. Complicating the situation is the presence of a roommate who does not contribute to the rent payments. This combination of factors, including the tenant's neurodevelopmental condition and the lack of financial support from their housemate, has led to the accrual of rent arrears and the potential risk of eviction.<|end|><|assistant|> No<|end|>"

In [16]:
dataset[0]

{'messages': [{'content': "Task: The tenant is a mid-forties individual with Autism spectrum disorders who lives in a three-story duplex they have occupied for over four years. Despite their recent interest in painting as a hobby, they have fallen behind on their $1,385 rent. Complicating the situation is the presence of a roommate who does not contribute to the rent payments. This combination of factors, including the tenant's neurodevelopmental condition and the lack of financial support from their housemate, has led to the accrual of rent arrears and the potential risk of eviction.",
   'role': 'user'},
  {'content': 'No', 'role': 'assistant'}]}

In [19]:
tokenizer.apply_chat_template(dataset[0]['messages'], tokenize=False)

"<s><|user|>\nTask: The tenant is a mid-forties individual with Autism spectrum disorders who lives in a three-story duplex they have occupied for over four years. Despite their recent interest in painting as a hobby, they have fallen behind on their $1,385 rent. Complicating the situation is the presence of a roommate who does not contribute to the rent payments. This combination of factors, including the tenant's neurodevelopmental condition and the lack of financial support from their housemate, has led to the accrual of rent arrears and the potential risk of eviction.<|end|>\n<|assistant|>\nNo<|end|>\n"

In [9]:
import torch
from transformers import Phi3ForSequenceClassification, AdamW
model_id = "microsoft/phi-3-mini-4k-instruct"

# Load the model
model = Phi3ForSequenceClassification.from_pretrained(model_id,
                                                           device_map='auto',
                                                           num_labels=2, 
                                                           torch_dtype=torch.bfloat16,
                                                           attn_implementation="flash_attention_2")

model.gradient_checkpointing_enable()
model.config.use_cache = False # Doesn't work with gradient checkpointing.

# Define the optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)

config.json:   0%|          | 0.00/904 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/16.3k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/2.67G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Some weights of Phi3ForSequenceClassification were not initialized from the model checkpoint at microsoft/phi-3-mini-4k-instruct and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [10]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Device: {device}")

Device: cuda


In [11]:
losses = []

model.train()
for epoch in range(1):  # Training for 3 epochs
    for batch in data_loader:
        input_ids, attention_mask, labels = batch['input_ids'].to(device), batch['attention_mask'].to(device), batch['labels'].to(device)

        # Forward pass
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        print(logits.shape)
        break

        # Calculate the loss
        loss_fct = torch.nn.CrossEntropyLoss()
        loss = loss_fct(logits, labels)
        losses.append(loss.detach().item())

        # Backward pass and optimization
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

    print(f"Epoch {epoch + 1}, Loss: {loss.item()}")

Epoch 1, Loss: 0.7578125
Epoch 2, Loss: 0.72265625
Epoch 3, Loss: 0.78125
Epoch 4, Loss: 0.9375
Epoch 5, Loss: 0.98828125
Epoch 6, Loss: 0.1064453125
Epoch 7, Loss: 0.006134033203125
Epoch 8, Loss: 0.11083984375
Epoch 9, Loss: 0.0003452301025390625
Epoch 10, Loss: 5.7220458984375e-05
Epoch 11, Loss: 5.054473876953125e-05
Epoch 12, Loss: 4.839897155761719e-05
Epoch 13, Loss: 4.482269287109375e-05


In [ ]:
import matplotlib.pyplot as plt 
plt.plot(losses)

In [ ]:
Dhat, _ = predict(model, data_loader, device)
Dhat = Dhat.reshape(-1,1)

In [ ]:
print(np.min(Dhat), np.max(Dhat))

In [ ]:
fig = plt.figure(dpi=300, tight_layout=True, figsize=(7, 4.5))
ax = plt.axes(facecolor=(.95, .96, .97))
ax.set_title('Probability of Treatment Given (Instrument==0)', size=16, pad=20)

# Set visibility of some spines
for key in 'left', 'right', 'top':
    ax.spines[key].set_visible(False)

# Add text to the axes
ax.text(0., 1.02, s='Density', transform=ax.transAxes, size=14)

# Set Y-axis tick parameters and grid
ax.yaxis.set_tick_params(length=0)
ax.yaxis.grid(True, color='white', linewidth=2)
ax.set_axisbelow(True)

# Plot the KDE plot
sns.kdeplot(Dhat[df['Instrument'].values==0].reshape(-1,), color='#36454F', fill=False, bw_adjust=0.25)
plt.xlim(0, 1)
plt.ylabel('')
plt.show()